In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Average
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

batch_size = 128
nb_classes = 2
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 20, 20
# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
input_shape=(20, 20,1)

Using TensorFlow backend.
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [2]:

vetor_atendente_final=pd.read_excel('Atendentes_Vetores.xlsx',header=0,index=False)
vetor_atendente_final.shape

(2771, 401)

In [3]:
vetor_cliente_final=pd.read_excel('Clientes_Vetores.xlsx',header=0,index=False)
vetor_atendente_final.iloc[:,0:-1].shape

(2771, 400)

In [4]:
target=np.array(pd.get_dummies(vetor_atendente_final.iloc[:,-1]))
target_=target
target

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [63]:
def norm2(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [82]:
tabela_dados=pd.read_excel('TabelaDados.xlsx',header=0,index=False)
tabela_dados2=tabela_dados.iloc[np.concatenate([np.where(target.T[1]==1)[0][0:1680],np.where(target.T[1]==0)[0][-1091:]],axis=0),:].fillna(0.9)
tabela_dados2['Satisfação cliente']=tabela_dados2['Satisfação cliente'].astype('category').cat.codes
tabela_dados2['Satisfação atendente']=tabela_dados2['Satisfação atendente'].astype('category').cat.codes
tabela_dados3=tabela_dados2[['Silencio','Tempo de Ligação','Atend_Lex','Cliente_Lex','Satisfação cliente',
                                     'Satisfação atendente']]
for i in tabela_dados3.columns[0:4]:
    tabela_dados3[i] = norm2(tabela_dados3[i])


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [83]:
tabela_dados3['Silencio']

0       0.700646
1       0.701806
2       0.062971
3       0.533697
4       0.068154
          ...   
2766    0.704984
2767    0.067725
2768    0.697560
2769    0.678057
2770    0.679891
Name: Silencio, Length: 2771, dtype: float64

In [84]:
bins = [0, 15, 25, 35, 50, 75,85, 100]
labels = [1,2,3,4,5,6,7]
for i in tabela_dados3.columns[0:4]:
    tabela_dados3[i] = pd.cut(tabela_dados3[i], bins=bins,labels=labels)
tabela_dados3['Silencio']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0       1
1       1
2       1
3       1
4       1
       ..
2766    1
2767    1
2768    1
2769    1
2770    1
Name: Silencio, Length: 2771, dtype: category
Categories (7, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7]

In [85]:
#tabela_dados3=np.array(tabela_dados3.iloc[:,[2,3]])

variaveis=tabela_dados3.shape[1]
concat=np.array(tabela_dados3)
concat=concat.reshape(-1,variaveis,1)
concat;

In [86]:
def norm(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

x_train0=np.array(norm(np.array(vetor_atendente_final.iloc[:,0:-1]))).reshape(-1,20,20,1)
x_train1=np.array(norm(np.array(vetor_cliente_final.iloc[:,0:-1]))).reshape(-1,20,20,1)
x_train1.shape

(2771, 20, 20, 1)

In [87]:
from sklearn.model_selection import train_test_split

X, y = x_train0, target
X1, y1 = x_train1, target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.1, random_state=42)
concat_train,concat_test, y_train2, y_test2=train_test_split(concat, y1, test_size=0.1, random_state=42)

In [89]:
from keras.layers import Input, Dense, Lambda, Concatenate
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.layers.core import Reshape
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D,Convolution1D,MaxPooling1D
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import os
from keras.optimizers import SGD
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


with session.as_default():
     with session.graph.as_default():
        
        
        

        input_shape2=(20,20,1)
        input_img = Input(batch_shape=(None, 20,20,1))
        input_img1 = Input(batch_shape=(None, 20,20,1))
        input_img2 = Input(batch_shape=(None, variaveis,1))


        denoise_left=Convolution2D(20, 3,3,
                                border_mode='same',
                                input_shape=input_shape)(input_img)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=MaxPooling2D(pool_size=(2,2))(denoise_left)
        denoise_left=Convolution2D(20, 3, 3,
                                    init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=UpSampling2D(size=(2, 2))(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=UpSampling2D(size=(2, 2))(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Convolution2D(1, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('sigmoid')(denoise_left)
        denoise_left=Dense(20)(denoise_left)
        denoise_left=Reshape((-1,20,1))(denoise_left)
        denoise_left=Flatten()(denoise_left)

        denoise_left1=Convolution2D(20, 3,3,
                                border_mode='same',
                                input_shape=input_shape)(input_img1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=MaxPooling2D(pool_size=(2,2))(denoise_left1)
        denoise_left1=Convolution2D(20, 3, 3,
                                    init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=UpSampling2D(size=(2, 2))(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=UpSampling2D(size=(2, 2))(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Convolution2D(1, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('sigmoid')(denoise_left1)
        denoise_left1=Dense(20)(denoise_left1)
        denoise_left1=Reshape((-1,20,1))(denoise_left1)
        denoise_left1=Flatten()(denoise_left1)

        denoise_concat=Dense(20)(input_img2)
        denoise_concat=Activation('relu')(denoise_concat)
        denoise_concat=Dense(20)(denoise_concat)
        denoise_concat=Activation('sigmoid')(denoise_concat)
        denoise_concat=Dense(variaveis)(denoise_concat)
        denoise_concat=Reshape((-1,variaveis,1))(denoise_concat)
        denoise_concat=Flatten()(denoise_concat)

        squeeze0=Concatenate()([denoise_left,denoise_left1,denoise_concat])
        squeeze0=Dropout(0.1)(squeeze0)
        squeeze0=Dense(20)(squeeze0)
        squeeze0=Activation('relu')(squeeze0)
        squeeze0=Dense(2)(squeeze0)
        squeeze0=Activation('sigmoid')(squeeze0)

        model = Model(inputs = [input_img,input_img1,input_img2], outputs = squeeze0)

        model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01,momentum=0.6, decay=5e-5, nesterov=False),metrics = ['accuracy'])
        save = ModelCheckpoint('pesos_keras.hdf5', save_best_only=True, monitor='val_loss', mode='min')

        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        session.run(init)
        model.fit([X_train, X_train1,concat_train],y_train,
                nb_epoch=2000,
                batch_size=2771,verbose=1,validation_split=0.1,callbacks=[save]
                 )
        preds=model.predict([X_test,X_test1,concat_test])

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), input_shape=(20, 20, 1..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), kernel_initializer="glorot_uniform", padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), kernel_initializer="glorot_uniform", padding="same")`
/usr/local/lib/python3.5/

Train on 2243 samples, validate on 250 samples
Epoch 1/2000
2243/2243 [==============================] - 3s 1ms/step - loss: 0.6908 - acc: 0.5305 - val_loss: 0.6858 - val_acc: 0.5800
Epoch 2/2000
2243/2243 [==============================] - 0s 78us/step - loss: 0.6708 - acc: 0.6068 - val_loss: 0.6964 - val_acc: 0.5800
Epoch 3/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6774 - acc: 0.6068 - val_loss: 0.6788 - val_acc: 0.5800
Epoch 4/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.6714 - acc: 0.6068 - val_loss: 0.6785 - val_acc: 0.5800
Epoch 5/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6714 - acc: 0.6068 - val_loss: 0.6805 - val_acc: 0.5800
Epoch 6/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6697 - acc: 0.6068 - val_loss: 0.6824 - val_acc: 0.5800
Epoch 7/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6699 - acc: 0.6068 - val_loss: 0.6798 - val_acc: 0.

2243/2243 [==============================] - 0s 77us/step - loss: 0.6367 - acc: 0.6701 - val_loss: 0.7451 - val_acc: 0.5800
Epoch 61/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6757 - acc: 0.6068 - val_loss: 0.6971 - val_acc: 0.4800
Epoch 62/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.7048 - acc: 0.4128 - val_loss: 0.6950 - val_acc: 0.4360
Epoch 63/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6979 - acc: 0.4062 - val_loss: 0.6815 - val_acc: 0.5800
Epoch 64/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.6797 - acc: 0.6077 - val_loss: 0.6695 - val_acc: 0.5800
Epoch 65/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6592 - acc: 0.6068 - val_loss: 0.6836 - val_acc: 0.5800
Epoch 66/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6598 - acc: 0.6068 - val_loss: 0.6875 - val_acc: 0.5800
Epoch 67/2000
2243/2243 [=======================

Epoch 120/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.7040 - acc: 0.4267 - val_loss: 0.6533 - val_acc: 0.7200
Epoch 121/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6756 - acc: 0.5684 - val_loss: 0.6547 - val_acc: 0.5800
Epoch 122/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6137 - acc: 0.6068 - val_loss: 0.7685 - val_acc: 0.5800
Epoch 123/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6489 - acc: 0.6068 - val_loss: 0.6871 - val_acc: 0.5800
Epoch 124/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6130 - acc: 0.6068 - val_loss: 0.6360 - val_acc: 0.5800
Epoch 125/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6285 - acc: 0.6072 - val_loss: 0.6613 - val_acc: 0.5800
Epoch 126/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.6078 - acc: 0.6077 - val_loss: 0.7111 - val_acc: 0.5800
Epoch 127/2000
2243/2243 [=

Epoch 179/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.5339 - acc: 0.7334 - val_loss: 0.7691 - val_acc: 0.5880
Epoch 180/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.5334 - acc: 0.7321 - val_loss: 0.7909 - val_acc: 0.5800
Epoch 181/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.5322 - acc: 0.7307 - val_loss: 0.7582 - val_acc: 0.5880
Epoch 182/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.5319 - acc: 0.7307 - val_loss: 0.8008 - val_acc: 0.5800
Epoch 183/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.5302 - acc: 0.7338 - val_loss: 0.7395 - val_acc: 0.5880
Epoch 184/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.5300 - acc: 0.7370 - val_loss: 0.8207 - val_acc: 0.5760
Epoch 185/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.5323 - acc: 0.7285 - val_loss: 0.7037 - val_acc: 0.6000
Epoch 186/2000
2243/2243 [=

...

Epoch 1713/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4735 - val_acc: 0.7080
Epoch 1714/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4712 - val_acc: 0.7080
Epoch 1715/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4731 - val_acc: 0.7080
Epoch 1716/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4695 - val_acc: 0.7080
Epoch 1717/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4759 - val_acc: 0.7080
Epoch 1718/2000
2243/2243 [==============================] - 0s 77us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4738 - val_acc: 0.7080
Epoch 1719/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4686 - val_acc: 0.7080
Epoch 1720/2000
2243

Epoch 1772/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 1.4804 - val_acc: 0.7080
Epoch 1773/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.4874 - val_acc: 0.7080
Epoch 1774/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 1.4810 - val_acc: 0.7080
Epoch 1775/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 1.4868 - val_acc: 0.7080
Epoch 1776/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 1.4885 - val_acc: 0.7080
Epoch 1777/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 1.4840 - val_acc: 0.7080
Epoch 1778/2000
2243/2243 [==============================] - 0s 76us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 1.4879 - val_acc: 0.7080
Epoch 1779/2000
2243

Epoch 1830/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.6247e-04 - acc: 1.0000 - val_loss: 1.5006 - val_acc: 0.7080
Epoch 1831/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.5956e-04 - acc: 1.0000 - val_loss: 1.4981 - val_acc: 0.7080
Epoch 1832/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.8257e-04 - acc: 1.0000 - val_loss: 1.4939 - val_acc: 0.7080
Epoch 1833/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.7130e-04 - acc: 1.0000 - val_loss: 1.4987 - val_acc: 0.7080
Epoch 1834/2000
2243/2243 [==============================] - 0s 77us/step - loss: 9.9764e-04 - acc: 1.0000 - val_loss: 1.4979 - val_acc: 0.7080
Epoch 1835/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.7738e-04 - acc: 1.0000 - val_loss: 1.5011 - val_acc: 0.7080
Epoch 1836/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.8697e-04 - acc: 1.0000 - val_loss: 1.5013 - val_acc:

Epoch 1887/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.2498e-04 - acc: 1.0000 - val_loss: 1.5043 - val_acc: 0.7080
Epoch 1888/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.9903e-04 - acc: 1.0000 - val_loss: 1.5083 - val_acc: 0.7080
Epoch 1889/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.2194e-04 - acc: 1.0000 - val_loss: 1.5033 - val_acc: 0.7080
Epoch 1890/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.2632e-04 - acc: 1.0000 - val_loss: 1.5068 - val_acc: 0.7080
Epoch 1891/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.2151e-04 - acc: 1.0000 - val_loss: 1.5072 - val_acc: 0.7080
Epoch 1892/2000
2243/2243 [==============================] - 0s 76us/step - loss: 9.1284e-04 - acc: 1.0000 - val_loss: 1.5031 - val_acc: 0.7080
Epoch 1893/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.9460e-04 - acc: 1.0000 - val_loss: 1.5058 - val_acc:

Epoch 1944/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.7364e-04 - acc: 1.0000 - val_loss: 1.5133 - val_acc: 0.7080
Epoch 1945/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.8012e-04 - acc: 1.0000 - val_loss: 1.5178 - val_acc: 0.7080
Epoch 1946/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.6983e-04 - acc: 1.0000 - val_loss: 1.5107 - val_acc: 0.7080
Epoch 1947/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.7027e-04 - acc: 1.0000 - val_loss: 1.5157 - val_acc: 0.7080
Epoch 1948/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.8088e-04 - acc: 1.0000 - val_loss: 1.5154 - val_acc: 0.7080
Epoch 1949/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.8751e-04 - acc: 1.0000 - val_loss: 1.5188 - val_acc: 0.7080
Epoch 1950/2000
2243/2243 [==============================] - 0s 76us/step - loss: 8.6296e-04 - acc: 1.0000 - val_loss: 1.5193 - val_acc:

In [90]:
from keras.utils import plot_model 
plot_model(model, to_file='model_gif.png')

In [91]:
model.load_weights('pesos_keras.hdf5')
loss,acc = model.evaluate([X_test, X_test1,concat_test],y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

278/278 [==============================] - 0s 155us/step
Restored model, accuracy: 78.06%


In [92]:
preds00=model.predict([X_test, X_test1,concat_test])
preds00

array([[0.01479644, 0.08452201],
       [0.04554692, 0.03810459],
       [0.01548803, 0.07909605],
       [0.03558069, 0.05155885],
       [0.01159286, 0.07654324],
       [0.1117295 , 0.03007555],
       [0.05435109, 0.03798705],
       [0.08941811, 0.0277718 ],
       [0.08082142, 0.02907884],
       [0.01741019, 0.08560592],
       [0.1254181 , 0.02996391],
       [0.01377845, 0.0712612 ],
       [0.09396726, 0.02609846],
       [0.04368818, 0.06020051],
       [0.12545586, 0.02757323],
       [0.01181287, 0.09053031],
       [0.05057061, 0.05010065],
       [0.04112312, 0.04037043],
       [0.06782243, 0.04215956],
       [0.05865076, 0.03648898],
       [0.04800981, 0.0544754 ],
       [0.01417953, 0.06169555],
       [0.01876035, 0.07655972],
       [0.02156082, 0.07494846],
       [0.02564999, 0.062929  ],
       [0.02706975, 0.0774287 ],
       [0.00988036, 0.0903078 ],
       [0.12448665, 0.03582695],
       [0.0562129 , 0.05613682],
       [0.03414282, 0.0502336 ],
       [0.

In [93]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,f1_score,recall_score


In [94]:
confusion_matrix(target,np.argmax(preds00,axis=1))
pred0=np.argmax(preds00,axis=1)
Y=target
print('\n')
print('Accuracy:',accuracy_score(np.array(Y),pred0))
print('Precision (FP):',precision_score(np.array(Y),pred0,average='binary'))
print('Recall (FN):',recall_score(np.array(Y),pred0,average='binary'))
print('f1:',f1_score(np.array(Y),pred0,average='binary'))
print(confusion_matrix(np.array(Y),pred0),'\n')



Accuracy: 0.7805755395683454
Precision (FP): 0.8054054054054054
Recall (FN): 0.8563218390804598
f1: 0.83008356545961
[[ 68  36]
 [ 25 149]] 

